**Title**: A short explicit title for that notebook  
**Date**:  DD-Mmm-YYYY  
**Description**:  
* This notebook is intended to provide a baseline structure to help standardizing how Flywheel presents public facing Jupyter notebooks.
* In general, Jupyter notebook should be structured as a tutorial that walks the user through one specific use case.
* It should be possible to run this notebook in any jupyter-compatible thrid party-platforms such as [google collab](https://colab.research.google.com/) or [mybinder.org](https://mybinder.org/).
* This template notebook is structured as follow:
    1. Three required sections are described: [Requirements](#Requirements), [Setup](#Setup) and [Flywheel API Key and Client](#Flywheel-API-Key-and-Client).
    2. Some [Suggestions on how to structure your notebook](#Suggestions-on-how-to-structure-your-notebook) are provided.
    3. A brief [Style Guide](#Style-Guide) is described.

# Requirements
- Detail here the minimum requirements for using this notebook.
- Examples: 
    - Access to Flywheel Instance
    - Flyhweel permission (i.e. admin, read/write, etc.)
    - Pre-requisites in terms of access to data
- Do not list here the required python packages as this is captured in a the [Setup](#Setup) section.

The next 2 sections ([Setup](#Setup) and [Flywheel API Key and Client](#Flywheel-API-Key-and-Client)) should be found in any notebook and kept as consistent as possible.

# <a id='setup'>Setup</a>

Packages required for the execution of this notebook should be installed with `pip` (using the `!` jupyter operator to run shell commands). This is required to ensure that the notebook is "standalone" and to avoid any issue with undefined package requirements. It also allows the notebook to be run out of the box on jupyter third party-platforms such as [google collab](https://colab.research.google.com/) or [mybinder.org](https://mybinder.org/).

In [ ]:
# Here is an example to install the flywheel SDK
!pip install flywheel-sdk

Once installed packages get imported. Import should first list Python standard packages and then Third-party packages.

In [ ]:
# Python standard package come first
from getpass import getpass
import logging
import os

# Third party packages come second
import flywheel
from permission import check_user_permission


If useful, a logger can be instantiated to display information during notebook execution (e.g. useful to keep track of runtime). 

In [ ]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO)
log = logging.getLogger('root')

# Flywheel API Key and Client

Tutorials based on Jupyter notebooks aim at illustrating interactions with a Flywheel instance using the Flywheel SDK.  
To communicate with a Flywheel instance your first need to authenticate with the Flywheel API which required getting an API_KEY for your account. You can get you API_KEY by following the steps described in the Flywheel SDK doc [here](https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/getting_started.html#api-key).

<div class="alert alert-block alert-danger">
<b>DANGER:</b> 
    Do NOT share your API key with anyone for any reason - it is the same as sharing your password and constitutes a HIPAA violation. ALWAYS obscure credentials from your code, especially when sharing with others/commiting to a shared repository.
</div>

In [ ]:
API_KEY = getpass('Enter API_KEY here: ')

Instantiate the Flywheel API client either using the API_KEY provided by the user input above or by reading it from the environment variable `FW_KEY`.

In [ ]:
fw = flywheel.Client(API_KEY if 'API_KEY' in locals() else os.environ.get('FW_KEY'))

You can check which Flywheel instance you have been authenticated against with the following:

In [ ]:
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

# Suggestions on how to structure your notebook

## Constants

Often you will have to define a few constants in your notebook which serve as the inputs. Such constant for instance is the API_KEY that was used to instantiate the Flywheel client. Other examples could be a PROJECT_ID or PROJECT_LABEL that will be used to identify a specific project.

In [ ]:
PROJECT_LABEL = 'MyProject'

# Requirements
Before getting started, you need to makesure the user has the right permission to run a specific actions like `run_gear` or `create_project_container` etc. 
We will be calling the `check_user_permission` function that we imported earlier to verify the user permission. 

First of all, we will define the minimum requirements needed in each container to run this notebook.

In [ ]:
min_reqs = {
"site": "user",
"group": "rw",
"project": ['files_download',
             'files_create_upload',
             'files_modify_metadata',
             'files_delete_non_device_data',
             'files_delete_device_data',]
}

The `check_user_permission` function takes 5 arguments: `fw_client` , `min_reqs`, `group_id`(if needed), `project_label`(if needed) and `show_compatible` (which is set default to `True`). In this example, since `PROJECT_LABEL` has been defined in the previous section, `GROUP_ID` needed to define before calling the function. 

In [ ]:
GROUP_ID = input('Please enter your Group ID here: ')


Here we will be calling the `check_user_permission` and  it will return True if both the group and project meet the minimum requirement, else a compatible list will be printed.

In [ ]:
check_user_permission(fw, min_reqs, group=GROUP_ID, project=PROJECT_LABEL)

## Helper functions

Your notebook may also need a few helper functions. We recommend that you defined them first in the notebook and use [Google Style Python docstrings](https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html#example-google-style-python-docstrings) to explicit what they are. Here is an example:

In [ ]:
def get_project_id(fw, project_label):
    """Return the first project ID matching project_label
    
    Args:
       fw (flywheel.Client): A flywheel client
       project_label (str):  A Project label
       
    Returns:
       (str): Project ID or None if no project found
    """
    project = fw.projects.find_first(f'label={project_label}')
    if project:
        return project.id
    else:
        return None

There is a few useful jupyter operator that can be leveraged when functions are defined that way. Such as diplaying the docstring of any function with the `?` operator or the source code itself with `??`.

In [ ]:
get_project_id?

## Main script

Once you have all the pieces defined, a short description of what the code does should be provided.

In this script, we will be retrieving the Project ID and a message will be printed to notify the user whether the project exist or not.

In [ ]:
project_id = get_project_id(fw, PROJECT_LABEL)
if project_id:
    print(f'Project ID is: {project_id}.')
else:
    print(f'No Project with label {PROJECT_LABEL} found.')

# Style Guide

In this section we present a few guidelines arround notebook styling.

## General recommendations

We recommend that best practices regarding Python styling be applied to notebook as well. In general, we follow the [PEP 8](https://www.python.org/dev/peps/pep-0008/) standard in writing Python code and adopted additional code standards like the [Google Python Style Guide](https://github.com/google/styleguide/blob/gh-pages/pyguide.md#google-python-style-guide).

## Boxes

Box are convenient to stress specific aspect or raise awarness of the user. Maintaining consistent color coding accross notebooks helps the user to better understand the intend of the box. Jupyter supports a few predefined CSS styling than can be used as follow:

<div class="alert alert-block alert-success">
<b>SUCCESS</b> Use this alert box indicates a milestone or information.
</div>

<div class="alert alert-block alert-info">
<b>INFO:</b> Use blue boxes for tips that can be useful for user to know.</div>

<div class="alert alert-block alert-warning">
<b>WARNING:</b> Use yellow boxes to warn the user on specific topic.
</div>

<div class="alert alert-block alert-danger">
<b>DANGER:</b> Use red boxes to indicate a dangerous or potentially negative action.
</div>

### Embedded Link

<div class="alert alert-block alert-info">
    <b>TIP:</b> If you want to embedded a link inside the box, please modify the text color to black, like this <a href="#" style="color:black">example link</a>.
</div>

## Capitalization

When mentioning Flywheel Container such as Project, Session, and Subject, always capitalize the first letter of the container name. 

Example:  
Each of Project, Subject, Session and Acquisition are containers. Containers shared common properties such as the ability to store files, metadata or analyses.

## Embed Image

When inserting images in markdown, we recommend to embed the image by using its URL path. 

Example:
<img src="https://gitlab.com/flywheel-io/public/flywheel-tutorials/-/raw/master/python/assets/anxiety_project_modify_metadata_w:csv-file.png" align="center"/>
